# Model AGB Jan
---

## Imports

In [1]:
import yt
import vtk
import h5py
import meshio
import matplotlib.pyplot as plt
import numpy             as np
import magritte.setup    as setup
import magritte.mesher   as mesher
import magritte.core     as magritte

from tqdm                   import tqdm
from astropy                import units, constants
from vtk.util.numpy_support import vtk_to_numpy
from scipy.spatial          import Delaunay, cKDTree
from scipy.interpolate      import griddata
from numba                  import njit
from yt.funcs               import mylog
mylog.setLevel(40) # This sets the log level to "ERROR"
from ipywidgets             import Dropdown, interact
from palettable.cubehelix   import cubehelix2_16
from matplotlib.gridspec    import GridSpec
from matplotlib             import rcParams, rc
from glob                   import glob

## Model level populations

In [2]:
model_file = 'model_Jan_reduced.hdf5'

In [3]:
# Load model
model = magritte.Model (model_file)

In [4]:
# Some setup
model.compute_spectral_discretisation ()
model.compute_inverse_line_widths     ()
model.compute_LTE_level_populations   ()

0

In [5]:
# Compute level populations from statistical equilibrium
# model.compute_level_populations (False, 1)

## Create synthetic observations

In [6]:
fcen = model.lines.lineProducingSpecies[0].linedata.frequency[0]
vpix = 1000
dd = vpix * (model.parameters.nfreqs()-1)/2 / magritte.CC
fmin = fcen - fcen*dd
fmax = fcen + fcen*dd

model.compute_spectral_discretisation (fmin, fmax)
model.compute_image (model.parameters.hnrays()-1)

imx = np.array(model.images[-1].ImX)
imy = np.array(model.images[-1].ImY)
imI = np.array(model.images[-1].I)
imv = np.array(model.radiation.frequencies.nu)[0]

In [7]:
# Extract the number of frequency bins
nfreqs = model.parameters.nfreqs()

# Set number of pixels in x and y direction
ngridx = 300
ngridy = 300

# Set a zoom factor
zoom = 1.3

# Set image boundaries
x_min, x_max = np.min(imx)/zoom, np.max(imx)/zoom
y_min, y_max = np.min(imy)/zoom, np.max(imy)/zoom

# Create image grid values
xs = np.linspace(x_min, x_max, ngridx)
ys = np.linspace(y_min, y_max, ngridy)

# Extract the spectral / velocity data
freqs = np.array(model.radiation.frequencies.nu)[0]
f_ij  = np.mean(freqs)
velos = (freqs - f_ij) / f_ij * constants.c.si.value * 1.0e-3   # [km/s]

# Interpolate the scattered data to an image (regular grid)
Is = np.zeros((nfreqs))
zs = np.zeros((nfreqs, ngridx, ngridy))
for f in range(nfreqs):
    zs[f] = griddata((imx, imy), imI[:,f], (xs[None,:], ys[:,None]), method='nearest')
    Is[f] = np.sum(zs[f])
Is = Is / np.max(Is)

# Get the logarithm of the data (matplotlib has a hard time handling logarithmic data.)
log_zs     = np.log(zs)
log_zs_min = np.min(log_zs)
log_zs_max = np.max(log_zs)

In [8]:
au = (1.0 * units.au).si.value

figs = []
gs   = GridSpec(1,2, wspace=.1, width_ratios=[2, 1])

for f in tqdm(range(nfreqs)):
    fig = plt.figure(dpi=300)
    ax1 = fig.add_subplot(gs[0])
    ax1.contourf(xs/au, ys/au, log_zs[f], cmap=cubehelix2_16.mpl_colormap, vmin=log_zs_min, vmax=log_zs_max, levels=250)
    ax1.set_aspect('equal')
    ax1.set_xlabel('image x [au]', labelpad = 10)
    ax1.set_ylabel('image y [au]', labelpad = 10)
    
    ax2 = fig.add_subplot(gs[1])
    ax2.plot(velos, Is/np.max(Is))
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    ax2.axvline(velos[f], c='red')
    ax2.set_ylabel('Relative intensity', labelpad=15)
    ax2.set_xlabel('velocity [km/s]',    labelpad=10)
    asp = 2*np.diff(ax2.get_xlim())[0] / np.diff(ax2.get_ylim())[0]
    ax2.set_aspect(asp)
    
    plt.savefig(f"images/image_{f:0>3d}.png", bbox_inches='tight')
    
    figs.append(fig)
    
    plt.close()
    
# Crete interactive figure
interact(lambda q: figs[q], q=(0, len(figs)-1))

100%|██████████| 101/101 [01:51<00:00,  1.10s/it]


interactive(children=(IntSlider(value=50, description='q'), Output()), _dom_classes=('widget-interact',))

<function __main__.<lambda>(q)>